In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

pd.set_option("mode.chained_assignment", None)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as backend_inline

backend_inline.set_matplotlib_formats("svg")

# Uciml repo
from ucimlrepo import fetch_ucirepo

# Pytorch device specific configuration
# Pytorch Gpu Configuration for cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pytorch Gpu Configuration for dml devices
# import torch_directml

# device = torch_directml.device()

# Set default devices
torch.set_default_device(device)

In [72]:
# Import heart desease data
heart_disease = fetch_ucirepo(id=45)

# Data as pandas dataframe
data = heart_disease.data.features
labels = heart_disease.data.targets

In [73]:
#### Data pre-processing ####
# Filling nan values with it's mean
keys = data.keys()
for key in keys:
    # print(f"{key} before filling na count: {data[key].isna().sum()}")
    data[key] = data[key].fillna(data[key].mean())
    # print(f"{key} after filling na count: {data[key].isna().sum()}")

# Binarized labels
labels = labels.values > 0
data = data.values
print(data.shape, labels.shape)

(303, 13) (303, 1)


In [ ]:
# Data normalization
data = data / np.max(data)
# convert data into tensors
dataT = torch.tensor(data).float()
labelsT = torch.tensor(labels).long()

# Train test split
train_data, test_data, train_labels, test_labels = train_test_split(
    dataT, labelsT, train_size=0.9
)

# Pytorch datasets
train_dataT = TensorDataset(train_data, train_labels)
test_dataT = TensorDataset(test_data, test_labels)

# Lead data into data loaders
batchSize = 16
train_loader = DataLoader(
    train_dataT,
    batch_size=batchSize,
    shuffle=True,
    drop_last=True,
    generator=torch.Generator(device),
)
test_loader = DataLoader(
    test_dataT,
    batch_size=test_dataT.tensors[0].shape[0],
    generator=torch.Generator(device),
)

In [ ]:
def createTheModel():
    class PredictHeartDisease(nn.Module):
        def __init__(self):
            super().__init__()

            # Input Layer
            self.input = nn.Linear(13, 128)
            # Hidden Layers
            self.hidden1 = nn.Linear(128, 256)
            self.hidden2 = nn.Linear(256, 256)
            self.hidden3 = nn.Linear(256, 128)
            # Output Layers
            self.output = nn.Linear(128, 1)

        def forward(self, x):
            x = F.relu(self.input(x))
            x = F.relu(self.hidden1(x))
            x = F.relu(self.hidden2(x))
            x = F.relu(self.hidden3(x))
            return self.output(x)

    net = PredictHeartDisease()

    # Loss Function
    lossFun = nn.BCEWithLogitsLoss()

    # Optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    return net, lossFun, optimizer

In [76]:
# Test the model for errors
net, lossFun, optimizer = createTheModel()
tempData = torch.randint(low=-1, high=2, size=(100, 13)).float()
net(tempData)

tensor([[0.0807],
        [0.0672],
        [0.0722],
        [0.0891],
        [0.0698],
        [0.0697],
        [0.0734],
        [0.0682],
        [0.0730],
        [0.0769],
        [0.0669],
        [0.0728],
        [0.0594],
        [0.0703],
        [0.0665],
        [0.0639],
        [0.0696],
        [0.0777],
        [0.0770],
        [0.0728],
        [0.0778],
        [0.0794],
        [0.0711],
        [0.0713],
        [0.0817],
        [0.0606],
        [0.0672],
        [0.0745],
        [0.0683],
        [0.0562],
        [0.0833],
        [0.0812],
        [0.0911],
        [0.0671],
        [0.0712],
        [0.0672],
        [0.0757],
        [0.0667],
        [0.0665],
        [0.0528],
        [0.0766],
        [0.0619],
        [0.0611],
        [0.0772],
        [0.0717],
        [0.0754],
        [0.0689],
        [0.0615],
        [0.0831],
        [0.0723],
        [0.0630],
        [0.0760],
        [0.0751],
        [0.0667],
        [0.0592],
        [0